In [ ]:
##Update the path and import libraries
import os
from os.path import isfile, join
import pandas as pd
path=r"C:\\Umass spring 20\\645\\github\\MLbasedDBMStuning\\Data\\Workloads\\"
outpath=r"C:\\Umass spring 20\\645\\github\\MLbasedDBMStuning\\Data\\Workloads\\offline\\"

In [ ]:
## Functions to return file name in directory
def loadWorkloadFileNames(folder):
    onlyfiles = [f for f in os.listdir(folder) if isfile(join(folder, f))]
    return onlyfiles

In [ ]:
#Read file to be segregated by workload
df_off=pd.read_csv('./Data/offline_workload.CSV')
print(df_off.shape)

In [ ]:
##Change knobs to boolean if not already
df_off['k7'] = df_off['k7'].apply(lambda x: 1 if x ==True else 0)
df_off['k7'] = df_off['k7'].apply(lambda x: 1 if x ==True else 0)

In [ ]:
## Convert workload Id to integer if not already
df_off['workload id']=df_off['workload id'].str.split('-')
df_off['workload id']=df_off['workload id'].str.join("")
df_off['workload id']=df_off['workload id'].astype(int)

workloads=list(df_off['workload id'].unique())


In [ ]:
## Segregating files by workloads
for i in workloads:
    temp=pd.DataFrame(columns=df_off.columns)
    for ind,r in df_off.iterrows():
        if int(r['workload id'])==i:
            #print (r.T)
            temp=temp.append(pd.Series(r[df_off.columns],index=df_off.columns ), ignore_index=True)
            #print(temp.shape)
            
    temp.to_pickle(outpath+ '\\workload_'+ str(i)+'.pkl')
    

In [ ]:
#Pruning the workloads by keeping only pruned metrics
pruned=['latency','driver.BlockManager.memory.onHeapMemUsed_MB.avg',
       'worker_1.Disk_transfers_per_second.dm-0',
       'driver.BlockManager.memory.maxMem_MB.avg',
       'driver.jvm.pools.Code-Cache.committed.avg_period',
       'driver.jvm.pools.Code-Cache.committed.avg_increase',
       'worker_1.Paging_and_Virtual_Memory.pgpgout',
       'executor.jvm.pools.PS-Old-Gen.committed.avg',
       'worker_1.Disk_Read_KB/s.sda',
       'driver.LiveListenerBus.queue.executorManagement.size.avg',
       'worker_2.Paging_and_Virtual_Memory.pgscan_direct_normal']
df=pd.DataFrame()
files=loadWorkloadFileNames(outpath)
for filename in files:
    if filename.endswith(".pkl"):
        
        df=pd.read_pickle(outpath+filename)
        #print(df.shape)
        knobs=list(df.columns[0:13])
        
        for i in df.columns:
            if i not in pruned and i not in knobs:
                df.drop(i,axis=1,inplace=True)
        print(df.shape)
        df.to_pickle(outpath++filename)
        df.to_csv(outpath+filename+'.csv',header=True,index=False)
